# 💬 Conversation Analysis - NotebookLM Podcast
**Complete Multi-Speaker Conversation Analysis Pipeline**

In [ ]:
# Cell 1: Setup
import json
import time
from datetime import datetime
from pathlib import Path

import torch
import torchaudio

device = "mps" if torch.backends.mps.is_available() else "cpu"
CONVERSATION_FILE = Path("data/audio/PRIMARY/UX_Psychology_From_Miller_s_Law_to_AI.m4a")

print(f"💬 Conversation Analysis: {CONVERSATION_FILE.name}")
print(f"Device: {device.upper()}")
print("✅ Setup complete")

In [ ]:
# Cell 2: Load LFM Model
from liquid_audio import ChatState, LFM2AudioModel, LFM2AudioProcessor

HF_REPO = "LiquidAI/LFM2.5-Audio-1.5B"

processor = LFM2AudioProcessor.from_pretrained(HF_REPO).eval()
model = LFM2AudioModel.from_pretrained(HF_REPO).eval()

if device != "cpu":
    model = model.to(device)

print(f"✅ Model loaded: {sum(p.numel() for p in model.parameters()):,} parameters")

In [ ]:
# Cell 3: Load Conversation Audio
def load_conversation_audio(audio_path, max_duration=300):
    waveform, sr = torchaudio.load(str(audio_path))

    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)

    max_samples = int(max_duration * sr)
    if waveform.shape[1] > max_samples:
        waveform = waveform[:, :max_samples]

    if sr != 24000:
        resampler = torchaudio.transforms.Resample(sr, 24000)
        waveform = resampler(waveform)
        sr = 24000

    return waveform, sr


waveform, sr = load_conversation_audio(CONVERSATION_FILE)
duration = waveform.shape[1] / sr

print(f"✅ Conversation loaded: {duration:.1f}s ({duration / 60:.1f} minutes)")

In [ ]:
# Cell 4: Conversation Transcription
chat = ChatState(processor)

chat.new_turn("system")
chat.add_text("Transcribe a conversation between multiple speakers.")
chat.end_turn()

chat.new_turn("user")
chat.add_audio(waveform, sr)
chat.end_turn()

chat.new_turn("assistant")

print("🎙️ Transcribing conversation...")
start_time = time.time()

transcribed_text = ""
for t in model.generate_sequential(**chat, max_new_tokens=2048):
    if t.numel() == 1:
        transcribed_text += processor.text.decode(t)

latency = time.time() - start_time

print(f"✅ Transcription complete: {latency:.1f}s")
print(f"Words: {len(transcribed_text.split())}")
print(f"Processing speed: {duration / latency:.2f}x real-time")

In [ ]:
# Cell 5: Display Results
print("📝 TRANSCRIPTION:")
print("=" * 50)
print(transcribed_text[:500] + "..." if len(transcribed_text) > 500 else transcribed_text)
print("=" * 50)

In [ ]:
# Cell 6: Analysis
words = transcribed_text.split()
sentences = transcribed_text.split(".")

print("📊 ANALYSIS:")
print(f"Words: {len(words)}")
print(f"Sentences: {len(sentences)}")
print(f"Speaking rate: {len(words) / (duration / 60):.1f} WPM")

In [ ]:
# Cell 7: Save Results
results = {
    "test_info": {
        "model": HF_REPO,
        "device": device,
        "timestamp": datetime.now().isoformat(),
    },
    "conversation_info": {
        "file": str(CONVERSATION_FILE),
        "duration_minutes": duration / 60,
    },
    "transcription_info": {
        "words": len(words),
        "processing_speed_realtime": duration / latency,
    },
}

results_path = Path("results")
results_path.mkdir(exist_ok=True)

with open(results_path / "conversation_results.json", "w") as f:
    json.dump(results, f, indent=2)

print("✅ Results saved")